# 근사 베이지안 계산

### (1) 변이 가설

이 장에서 다룰 것은 변이 가설이다. 이는 다음과 같다. <br />
"남자가 여자보다 능력 범위가 넓고, 특히 지적인 면에서 훨씬 뛰어나다고 주장한 조안 미켈이 19세기 초에 만든 분야이다. <br/>
그는 대부분의 천재와 대부분의 정신적 미숙아는 남자라고 믿었다. 그는 남자는 '우수한 동물'이라고 생각해서 <br />
여성의 변이 부족은 열등함의 표시라고 결론지었다."

여기서 만약 여성이 실제로 남자보다 가변적인 것이 드러나면 미켈 역시 이를 열등함의 표시라고 받아들여야 할 것이다. <br />

미국의 남녀 성인이 키를 직접 입력한 데이터를 살펴보자. <br />
데이터셋에는 15,507명의 남성과 254,722명의 여성의 응답 내용이 들어있다. <br />

이 데이터의 내용은 다음과 같다.
- 남성의 평균 키는 178cm이고, 여성의 평균 키는 163cm이다. 즉 평균적으로 남성이 더 크다.
- 남성의 표준편차는 7.7cm이고, 여성의 경우는 7.3cm이다. 따라서 남성의 키가 더 가변적이다.
- 하지만 그룹 간의 변이를 비교할때 표준편차를 평균으로 나누는 변동 계수(Coefficient of variation, CV)를 사용하면 <br />
남성의 경우 CV가 0.0433이고, 여성의 경우 0.0444이다. 이 값은 변이가 원 수치와 관계없는 무차원적 측정치이다.

숫자들이 매우 비슷하므로 데이터셋이 변이 가설을 증명하기에는 빈약하다고 결론을 내릴 수 있다. <br />
하지만 베이지안 메서드를 사용하면 결론을 더 명확하게 만들 수 있다.

#### 전체 코드

In [1]:
import math
import numpy
import pickle
import numpy
import random
import scipy

import brfss

import thinkplot
import thinkbayes2

import matplotlib.pyplot as pyplot


NUM_SIGMAS = 1

class Height(thinkbayes2.Suite, thinkbayes2.Joint):
    """Hypotheses about parameters of the distribution of height."""

    def __init__(self, mus, sigmas, label=None):
        """Makes a prior distribution for mu and sigma based on a sample.

        mus: sequence of possible mus
        sigmas: sequence of possible sigmas
        label: string label for the Suite
        """
        pairs = [(mu, sigma) 
                 for mu in mus
                 for sigma in sigmas]

        thinkbayes2.Suite.__init__(self, pairs, label=label)

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        Args:
            hypo: tuple of hypothetical mu and sigma
            data: float sample

        Returns:
            likelihood of the sample given mu and sigma
        """
        x = data
        mu, sigma = hypo
        like = scipy.stats.norm.pdf(x, mu, sigma)
        return like

    def LogLikelihood(self, data, hypo):
        """Computes the log likelihood of the data under the hypothesis.

        Args:
            data: a list of values
            hypo: tuple of hypothetical mu and sigma

        Returns:
            log likelihood of the sample given mu and sigma (unnormalized)
        """
        x = data
        mu, sigma = hypo
        loglike = EvalNormalLogPdf(x, mu, sigma)
        return loglike

    def LogUpdateSetFast(self, data):
        """Updates the suite using a faster implementation.

        Computes the sum of the log likelihoods directly.

        Args:
            data: sequence of values
        """
        xs = tuple(data)
        n = len(xs)

        for hypo in self.Values():
            mu, sigma = hypo
            total = Summation(xs, mu)
            loglike = -n * math.log(sigma) - total / 2 / sigma**2
            self.Incr(hypo, loglike)

    def LogUpdateSetMeanVar(self, data):
        """Updates the suite using ABC and mean/var.

        Args:
            data: sequence of values
        """
        xs = data
        n = len(xs)

        m = numpy.mean(xs)
        s = numpy.std(xs)

        self.LogUpdateSetABC(n, m, s)

    def LogUpdateSetMedianIPR(self, data):
        """Updates the suite using ABC and median/iqr.

        Args:
            data: sequence of values
        """
        xs = data
        n = len(xs)

        # compute summary stats
        median, s = MedianS(xs, num_sigmas=NUM_SIGMAS)
        print('median, s', median, s)

        self.LogUpdateSetABC(n, median, s)

    def LogUpdateSetABC(self, n, m, s):
        """Updates the suite using ABC.

        n: sample size
        m: estimated central tendency
        s: estimated spread
        """
        for hypo in sorted(self.Values()):
            mu, sigma = hypo

            # compute log likelihood of m, given hypo
            stderr_m = sigma / math.sqrt(n)
            loglike = EvalNormalLogPdf(m, mu, stderr_m)

            #compute log likelihood of s, given hypo
            stderr_s = sigma / math.sqrt(2 * (n-1))
            loglike += EvalNormalLogPdf(s, sigma, stderr_s)

            self.Incr(hypo, loglike)


def EvalNormalLogPdf(x, mu, sigma):
    """Computes the log PDF of x given mu and sigma.

    x: float values
    mu, sigma: paramemters of Normal

    returns: float log-likelihood
    """
    return scipy.stats.norm.logpdf(x, mu, sigma)


def FindPriorRanges(xs, num_points, num_stderrs=3.0, median_flag=False):
    """Find ranges for mu and sigma with non-negligible likelihood.

    xs: sample
    num_points: number of values in each dimension
    num_stderrs: number of standard errors to include on either side
    
    Returns: sequence of mus, sequence of sigmas    
    """
    def MakeRange(estimate, stderr):
        """Makes a linear range around the estimate.

        estimate: central value
        stderr: standard error of the estimate

        returns: numpy array of float
        """
        spread = stderr * num_stderrs
        array = numpy.linspace(estimate-spread, estimate+spread, num_points)
        return array

    # estimate mean and stddev of xs
    n = len(xs)
    if median_flag:
        m, s = MedianS(xs, num_sigmas=NUM_SIGMAS)
    else:
        m = numpy.mean(xs)
        s = numpy.std(xs)

    print('classical estimators', m, s)

    # compute ranges for m and s
    stderr_m = s / math.sqrt(n)
    mus = MakeRange(m, stderr_m)

    stderr_s = s / math.sqrt(2 * (n-1))
    sigmas = MakeRange(s, stderr_s)

    return mus, sigmas


def Summation(xs, mu, cache={}):
    """Computes the sum of (x-mu)**2 for x in t.

    Caches previous results.

    xs: tuple of values
    mu: hypothetical mean
    cache: cache of previous results
    """
    try:
        return cache[xs, mu]
    except KeyError:
        ds = [(x-mu)**2 for x in xs]
        total = sum(ds)
        cache[xs, mu] = total
        return total


def CoefVariation(suite):
    """Computes the distribution of CV.

    suite: Pmf that maps (x, y) to z

    Returns: Pmf object for CV.
    """
    pmf = thinkbayes2.Pmf()
    for (m, s), p in suite.Items():
        pmf.Incr(s/m, p)
    return pmf


def PlotCdfs(d, labels):
    """Plot CDFs for each sequence in a dictionary.

    Jitters the data and subtracts away the mean.

    d: map from key to sequence of values
    labels: map from key to string label
    """
    thinkplot.Clf()
    for key, xs in d.items():
        mu = thinkbayes2.Mean(xs)
        xs = thinkbayes2.Jitter(xs, 1.3)
        xs = [x-mu for x in xs]
        cdf = thinkbayes2.MakeCdfFromList(xs)
        thinkplot.Cdf(cdf, label=labels[key])
    thinkplot.Show()
                  

def PlotPosterior(suite, pcolor=False, contour=True):
    """Makes a contour plot.
    
    suite: Suite that maps (mu, sigma) to probability
    """
    thinkplot.Clf()
    thinkplot.Contour(suite.GetDict(), pcolor=pcolor, contour=contour)

    thinkplot.Save(root='variability_posterior_%s' % suite.label,
                title='Posterior joint distribution',
                xlabel='Mean height (cm)',
                ylabel='Stddev (cm)')


def PlotCoefVariation(suites):
    """Plot the posterior distributions for CV.

    suites: map from label to Pmf of CVs.
    """
    thinkplot.Clf()
    thinkplot.PrePlot(num=2)

    pmfs = {}
    for label, suite in suites.items():
        pmf = CoefVariation(suite)
        print('CV posterior mean', pmf.Mean())
        cdf = thinkbayes2.MakeCdfFromPmf(pmf, label)
        thinkplot.Cdf(cdf)
    
        pmfs[label] = pmf

    thinkplot.Save(root='variability_cv',
                xlabel='Coefficient of variation',
                ylabel='Probability')

    print('female bigger', thinkbayes2.PmfProbGreater(pmfs['female'],
                                                     pmfs['male']))
    print('male bigger', thinkbayes2.PmfProbGreater(pmfs['male'],
                                                   pmfs['female']))


def PlotOutliers(samples):
    """Make CDFs showing the distribution of outliers."""
    cdfs = []
    for label, sample in samples.items():
        outliers = [x for x in sample if x < 150]

        cdf = thinkbayes2.MakeCdfFromList(outliers, label)
        cdfs.append(cdf)

    thinkplot.Clf()
    thinkplot.Cdfs(cdfs)
    thinkplot.Save(root='variability_cdfs',
                title='CDF of height',
                xlabel='Reported height (cm)',
                ylabel='CDF')


def PlotMarginals(suite):
    """Plots marginal distributions from a joint distribution.

    suite: joint distribution of mu and sigma.
    """
    thinkplot.Clf()

    pyplot.subplot(1, 2, 1)
    pmf_m = suite.Marginal(0)
    cdf_m = thinkbayes2.MakeCdfFromPmf(pmf_m)
    thinkplot.Cdf(cdf_m)

    pyplot.subplot(1, 2, 2)
    pmf_s = suite.Marginal(1)
    cdf_s = thinkbayes2.MakeCdfFromPmf(pmf_s)
    thinkplot.Cdf(cdf_s)

    thinkplot.Show()


def ReadHeights(nrows=None):
    """Read the BRFSS dataset, extract the heights and pickle them.

    nrows: number of rows to read
    """
    resp = brfss.ReadBrfss(nrows=nrows).dropna(subset=['sex', 'htm3'])
    groups = resp.groupby('sex')

    d = {}
    for name, group in groups:
        d[name] = group.htm3.values

    return d


def UpdateSuite1(suite, xs):
    """Computes the posterior distibution of mu and sigma.

    Computes untransformed likelihoods.

    suite: Suite that maps from (mu, sigma) to prob
    xs: sequence
    """
    suite.UpdateSet(xs)


def UpdateSuite2(suite, xs):
    """Computes the posterior distibution of mu and sigma.

    Computes log likelihoods.

    suite: Suite that maps from (mu, sigma) to prob
    xs: sequence
    """
    suite.Log()
    suite.LogUpdateSet(xs)
    suite.Exp()
    suite.Normalize()


def UpdateSuite3(suite, xs):
    """Computes the posterior distibution of mu and sigma.

    Computes log likelihoods efficiently.

    suite: Suite that maps from (mu, sigma) to prob
    t: sequence
    """
    suite.Log()
    suite.LogUpdateSetFast(xs)
    suite.Exp()
    suite.Normalize()


def UpdateSuite4(suite, xs):
    """Computes the posterior distibution of mu and sigma.

    Computes log likelihoods efficiently.

    suite: Suite that maps from (mu, sigma) to prob
    t: sequence
    """
    suite.Log()
    suite.LogUpdateSetMeanVar(xs)
    suite.Exp()
    suite.Normalize()


def UpdateSuite5(suite, xs):
    """Computes the posterior distibution of mu and sigma.

    Computes log likelihoods efficiently.

    suite: Suite that maps from (mu, sigma) to prob
    t: sequence
    """
    suite.Log()
    suite.LogUpdateSetMedianIPR(xs)
    suite.Exp()
    suite.Normalize()


def MedianIPR(xs, p):
    """Computes the median and interpercentile range.

    xs: sequence of values
    p: range (0-1), 0.5 yields the interquartile range

    returns: tuple of float (median, IPR)
    """
    cdf = thinkbayes2.MakeCdfFromList(xs)
    median = cdf.Percentile(50)

    alpha = (1-p) / 2
    ipr = cdf.Value(1-alpha) - cdf.Value(alpha)
    return median, ipr


def MedianS(xs, num_sigmas):
    """Computes the median and an estimate of sigma.

    Based on an interpercentile range (IPR).

    factor: number of standard deviations spanned by the IPR
    """
    half_p = thinkbayes2.StandardNormalCdf(num_sigmas) - 0.5
    median, ipr = MedianIPR(xs, half_p * 2)
    s = ipr / 2 / num_sigmas

    return median, s

def Summarize(xs):
    """Prints summary statistics from a sequence of values.

    xs: sequence of values
    """
    # print smallest and largest
    xs.sort()
    print('smallest', xs[:10])
    print('largest', xs[-10:])

    # print median and interquartile range
    cdf = thinkbayes2.MakeCdfFromList(xs)
    print(cdf.Percentile(25), cdf.Percentile(50), cdf.Percentile(75))


def RunEstimate(update_func, num_points=31, median_flag=False):
    """Runs the whole analysis.

    update_func: which of the update functions to use
    num_points: number of points in the Suite (in each dimension)
    """
    d = ReadHeights(nrows=None)
    labels = {1:'male', 2:'female'}

    # PlotCdfs(d, labels)

    suites = {}
    for key, xs in d.items():
        label = labels[key]
        print(label, len(xs))
        Summarize(xs)

        xs = thinkbayes2.Jitter(xs, 1.3)

        mus, sigmas = FindPriorRanges(xs, num_points, median_flag=median_flag)
        suite = Height(mus, sigmas, label)
        suites[label] = suite
        update_func(suite, xs)
        print('MLE', suite.MaximumLikelihood())

        PlotPosterior(suite)

        pmf_m = suite.Marginal(0)
        pmf_s = suite.Marginal(1)
        print('marginal mu', pmf_m.Mean(), pmf_m.Var())
        print('marginal sigma', pmf_s.Mean(), pmf_s.Var())

        # PlotMarginals(suite)

    PlotCoefVariation(suites)


def main():
    random.seed(17)

    func = UpdateSuite5
    median_flag = (func == UpdateSuite5)
    RunEstimate(func, median_flag=median_flag)


if __name__ == '__main__':
    main()

male 154407
smallest [  61.   74.   76.   81.   86.   89.   89.   91.   97.  101.]
largest [ 218.  221.  221.  221.  221.  225.  226.  229.  229.  236.]
173.0 178.0 183.0
classical estimators 178.37680558 7.43067384489
median, s 178.37680558 7.43067384489
MLE (178.37680557998382, 7.4306738448888865)


/Users/choi/anaconda/lib/python3.6/site-packages/matplotlib/axes/_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


Writing variability_posterior_male.pdf
Writing variability_posterior_male.eps
marginal mu 178.37680558 0.000350420307418
marginal sigma 7.43071586209 0.000175210671709
female 254722
smallest [ 61.  61.  64.  66.  74.  81.  89.  89.  89.  91.]
largest [ 213.  213.  213.  213.  221.  226.  229.  229.  229.  229.]
157.0 163.0 168.0
classical estimators 163.346191385 7.15672683982
median, s 163.346191385 7.15672683982
MLE (163.34619138541342, 7.156726839817523)
Writing variability_posterior_female.pdf
Writing variability_posterior_female.eps
marginal mu 163.346191385 0.000197042736953
marginal sigma 7.15675137089 9.85215450402e-05
CV posterior mean 0.04165741123
CV posterior mean 0.0438133963397
Writing variability_cv.pdf
Writing variability_cv.eps
female bigger 1.0000000000000033
male bigger 0


### (2) 평균과 표준편차
9장에서 두 변수를 결합 분포를 통해 동시에 추정하였다. <br />
이 장에서는 가우시안 분포의 변수인 평균 mu와 표준편차 sigma를 동일한 방식으로 추정할 것이다.

이 문제에서는 각 mu와 sigma의 쌍과 이 쌍이 나타날 확률의 연결 내용을 만드는 Heught Suite를 정의하겠다. 

In [ ]:
class Height(thinkbayes2.Suite, thinkbayes2.Joint):
    def __init__(self, mus, sigmas):
        thinkbayes2.Suite.__init__(self)
        
        pairs = [(mu, sigma) 
                 for mu in mus
                 for sigma in sigmas]
        
        thinkbayes.Suite.__init__(self, pairs)

mus는 mu의 가능한 값의 연속이고, sigmas는 sigma의 값의 연속이다. 모든 mu, sigma 쌍의 사전 분포는 균등 분포다.

mu와 sigma의 가설 값이 주어졌을 때 특정 값 x에 대한 우도를 계산할 수 있다. <br /> 
이 기능은 scipy.stats.norm.pdf()가 수행한다.

In [ ]:
# class Height

    def Likelihood(self, data, hypo):
        x = data
        mu, sigma = hype
        like = scipy.stats.norm.pdf(x, mu, sigma)
        return like

우리의 목적에서 우리가 원하는 것은 확률의 비율이다. 이런 목적으로는 확률 밀도가 적절하다. <br />
이 문제의 가장 어려운 부분은 mus와 sigmas의 적절한 범위를 고르는 일이다. <br />
범위가 너무 작으면 몇가지 무시해서는 안 될 확률 값을 지나칠 수 있고, <br />
범위가 너무 크면 답은 맞겠지만 쓸데없이 연산 능력을 낭비할 수 있다.

따라서 베이지안 기술을 더 효율적으로 쓰기 위해 고전적 추정 방법을 사용한다. <br />
mu와 sigma의 적절한 위치를 찾는데 고전적 추정치를 사용하고, 이 추정값의 적절한 분포를 선택하기 위해 표준편차를 사용한다.

분포의 진짜 변수가 mu와 sigma이고, 여기서 n개의 샘플을 취할 때 mu의 추정치는 샘플의 평균인 m이다. <br />
또한 sigma의 추정치는 샘플의 표준편차인 s다. 추정된 mu의 표준편차는 s/root(n)이고 <br />
sigma의 추정치인 표준 오차는 s/root(2(n-1)) 이다.

In [ ]:
def FindPriorRange(xs, num_points, num_stderrs=3.0):
    
    # m과 s를 계산함
    
    n = len(xs)
    m = numpy.me(xs)
    s = numpy.std(xs)
    
    # m과 s의 범위를 계산
    
    stderr_m = s / math.sqrt(n)
    mus = MakeRange(m, stderr_m)
    
    stderr_s = s / marh.sqrt(2*(n-1))
    sigmas = MakeRange(s, stderr_s)
    
    return mus, sigmas    

xs는 데이터셋이다. num_points는 범위 내의 필요한 값의 갯수다. num_stderrs는 표준 오차 수 내의 추정치의 양의 값 범위다.<br />
결과 값은 연속값 mus와 sigmas의 쌍이다.

다음 코드는 MakeRange다.

In [ ]:
    def MakeRange(estimate, stderr):
        spread = stderr*num_stderrs
        array = numpy.linspace(estimate - spread,
                               estimate + spread,
                               num_points)
        return array

num.linspace는 estimate-spread와 estimate+spread사이에 이 두 값을 포함하여 간격이 동일한 원소들의 행렬이다.

### (3) 갱신

마지막으로 Suite를 만들고 갱신하는 코드이다.

In [ ]:
    mus, sigmas = FindPriorRanges(xs, num_points)
    suite = Height(mus, sigmas)
    suite.UpdateSet(xs)
    print (suite.MaximumLikelihood())

우리는 사전 분포에서 데이터의 범위를 선택한 후, 갱신한 데이터를 다시 사용한다. <br />
보통 같은 데이터를 두 번 사용하면 안된다. 하지만 이 경우에는 괜찮다. <br />
이유는 수 많은 매우 작은 값에 대해서는 계산하지 않기 위해서이다.
num_stderr = 4 일때 범위는 무시하면 안 되는 우도에 대한 모든 값을 포함할 정도로 충분히 크다. <br />
따라서 좀 더 커진다고 해도 결과에는 아무런 영향을 끼치지 않는다. <br />
실제로는, 사전 분포는 mu와 sigma의 모든 값에 대해 균등 분포이지만, 계산의 효율성을 위해 필요없는 값들은 무시한다.

### (4) CV의 사후 분포

In [ ]:
def CoefVariation(suite):
    pmf = thinkbayes2.Pmf()
    for (mu, sigma), p in suite.Items():
        pmf.Incr(sigma/mu, p)
    return pmf

그 후 thinkbayes2.PmfProbGreater를 사용해서 남자가 보다 변이성이 높은 확률을 계산할 것이다.

### (5) 언더플로

우도 계산에 확률 밀도를 사용하는데, 연속형 분포의 경우 밀도가 매우 작기 때문에 이를 서로 곱하면 결과는 매우 작아진다. <br />
이를 언더플로(Under flow)라고 한다. 

이를 위한 해법으로 로그 변환하에서 우도를 계산하는 방법이 있다. <br />
이 방법에서는 작은 값들을 곱하는 대신 로그 우도를 더한다. <br />
Log는 Pmf내의 확률 로그를 계산한다.

In [ ]:
# Class Pmf
    def Log(self):
        m = self.MaxLike()
        for x, p in self.d.iteritems():
        if p:
            self.Set(x, math.log(p/m))
        else:
            self.Remove(x)

로그 변환 전에 Log는 MaxLike를 사용해서 Pmf에서 가장 높은 확률 m을 찾는다. <br />
이 메서드에서는 모든 확률을 m으로 나누므로 가장 높은 확률이 log0인 1로 정규화된다. <br />
다른 로그 확률은 모두 음수가 된다. 만약 0인 Pmf값이 있다면 이 값은 제거된다.

Pmf가 로그 변환된 상태에서는 Update, UpdateSet, Normalize를 사용할 수 없다. <br />
따라서 변형된 메서드인 LogUpdate와 LogUpdateSet을 사용해야 한다. <br />
다음은 LogUpdateSet을 구현한 내용이다.

In [ ]:
# class Suite
    
    def LogUpdateSet(self, dataset):
        for data in dataset:
            self.LogUpdate(data)

In [ ]:
LogUpdateSet은 데이터를 따라 반복하면서 LogUpdate를 호출한다.

In [ ]:
# class Suite

    def LogUpdate(self, data):
        for hypo in self.Values():
            like = self.LogLikelihood(data, hypo)
            self.Incr(hypo, like)

LogUpdate는 Likelihood 대신 LogLikelihood를 호출하고, Mult 대신 Incr을 호출한다는 점을 제외하면 Update와 유사하다.

In [ ]:
로그 우도를 사용하면 언더플로에서 문제가 생기지 않지만, 로그 변환하에서는 Pmf로 더 이상 할 수 있는 것이 없다. <br />
따라서 이미 변환된 것을 Exp를 사용해서 다시 원래대로 되돌린다. 

In [ ]:
# class Pmf
    
    def Exp(self):
        m = self.MaxLike()
        for x, p in self.d.iteritems():
            self.set(x, math.exp(p-m))

만약 로그 우도가 매우 큰 음수라면, 결과 우도는 언더플로일 것이므로 Exp가 최대 로그 우도 m을 찾아서 <br />
m을 찾아서 모든 우도를 m을 사용해 상향 이동시킨다. 결과 분포는 1에 대한 최대 우도를 갖는다. <br />
이 프로세스는 로그 변환된 내용을 정확도 손실을 최소로 해서 원래 상태로 되돌린다.

### (6) 로그 우도

다음은 LogLikelihood이다.

In [ ]:
# class Height
    
    def LogLikelihood(self, data, hypo):
        x = data
        mu, sigma = hypo
        loglike = scipy.stats.norm.logpdf(x, mu, sigma)
        return loglike

norm.logpdf는 가우시안 PDF의 로그 우도를 계산한다. 다음은 전체 갱신 프로세스의 내용이다.

정리해보면 Log는 로그 변환하에서 스윗을 실행한다. LogUpdateSet은 LogUpdate를 호출하고 이 함수는 LogLikelihood를 호출한다. <br />
LogUpdate는 Pmf.Incr을 사용하여 로그 우도를 더함으로써 우도를 곱하는 것과 동일한 결과를 낸다. <br />
갱신 후에는 로그 우도가 매우 큰 음수가 되므로 언더플로가 되는 것을 방지하기 위해 원상 복구하기 전에 Exp를 사용하여 상향 이동시킨다. <br />
스윗이 원상 복구되면, 확률은 다시 선형으로 되어 Normalize를 사용할 수 있다. 

### (7) 약간의 최적화

Suite.LogUpdateSet은 각 데이터 지점에 대해 한 번씩 LogUpdate를 호출한다. <br /> 
이를 전체 데이터셋에 대해 우도를 한 번에 계산하면 속도를 계선할 수 있다. 

가우시안 PDF에서 시작해보자.

<img src="./Images/1.png" width=300 />

그리고 (상수항에 대한) 로그를 계산한다. 

<img src="./Images/2.png" width=300 />

주어진 값 x(i)의 연속항에 대해, 전체 로그 우도는 다음과 같다. 

<img src="./Images/3.png" width=300 />

i에 의존하지 않는 항목들을 밖으로 꺼내면 다음과 같은 식이 나온다.

<img src="./Images/4.png" width=300 />

이 식을 파이썬으로 변환하면 다음과 같다.

In [ ]:
# class Height
    
    def LogUpdateSetFast(self, data):
        xs = tuple(data)
        n = len(xs)
        
        for hypo in self.Values():
            mu, sigma = hypo
            total = Summation(xs, mu)
            loglike = -n*math.log(sigma) - total / 2 / sigma**2
            self.Incr(hypo, loglike)

이 자체로도 약간의 개선이 있지만 더 개선을 원한다면 <br />
덧셈은 mu에만 좌우될 뿐, sigma와는 상관이 없다는 것을 기억하면, mu의 값만 계산해도 된다는 것을 알 수 있다. <br />
다시 계산하는 것을 피하기 위해, 덧셈을 하는 함수를 만든 후 메모이즈해서 사전에 게산된 결과가 딕셔너리에 저장되도록 한다.

> ### Cf) 메모이제이션
>  컴퓨터 프로그램이 동일한 계산을 반복해야 할 때, 이전에 계산한 값을 메모리에 저장함으로써 <br />
> 동일한 계산의 반복 수행을 제거하여 프로그램 실행 속도를 빠르게 하는 기술

In [ ]:
def Summation(xs, cache={}):
    try:
        return cache[xs, mu]
    except KeyError:
        ds = [(x-mu)**2 for x in xs]
        total = sum(ds)
        cache[xs, mu] = total
        return total

cache는 전에 계산된 합을 저장한다. try 구문은 가능한 경우 캐시의 값을 반환하고, 아닌 경우 합을 계산한 후 캐시에 넣고 결과를 반환한다. <br />
여기서 유일한 단점은 캐시에서는 키로 리스트를 사용할 수 없다는 점인데, 이는 해시가 가능한 타입이 아니기 때문이다. <br /> 그래서 LogUpdateSet에서 데이터를 튜플로 만든다.

### (8) 근사 베이지안 계산(ABC)

ABC(Approximate Baysian Computation)은 어떤 특정 데이터셋의 우도가 다음과 같을 때 필요하다. <br />
- 특정 큰 데이터셋의 경우, 매우 작아서 로그 변환이 어려운 경우
- 계산 비용이 많이 들어서 최적화를 많이 해야 하는 경우

우리는 우리가 본 정확한 데이터셋을 보는 것에 대한 우도가 어떻든 별로 상관없다. <br />
특히 연속형 변수의 경우, 우리가 본 것과 같은 모든 데이터셋을 보게 되는 것에 대한 우도에만 신경 쓴다. <br />

예를들어, 유로 문제에서 우리는 동전을 던진 순서에 대해서는 전혀 신경 쓰지 않고, 앞면과 뒷면의 전체 수에만 관심을 두었다. <br />
기차 문제의 경우 어떤 특정 기차를 보았는지는 상관없이 기차의 수와 번호의 최댓값에만 신경을 썻다.

따라서 이렇게 묻는게 더 정확할 것이다. <br />
"만약 mu와 sigma를 가설 값으로 갖는 인구 중 100,000명의 사람을 샘플링하면 <br /> 
관측 평균과 분산에 대한 샘플을 모을 수 있는 확률이 얼마나 될까?"

가우시안 분포에서 샘플링하면 이미 샘플 통계에서 분포를 정확히 찾아낼 수 있기 때문에 보다 효과적으로 답을 알아낼 수 있다. <br />
만약 변수 mu와 sigma를 사용하는 가우시안 분포에서 n개의 값을 뽑아내서 샘플 평균 m을 구한 경우 <br />
m의 분포는 mu와 sigma / root(n)을 변수로 사용하는 가우시안 분포를 따른다.

유사하게, 샘플 표준편차 s의 분포는 sigma / root(2(n-1))의 변수를 취하는 가우시안 분포다. <br />
이 샘플 분포를 사용해서 mu와 sigma의 가설 값이 주어졌을 때 샘플 통계의 우도를 계산 할 수 있다. 

다음은 이를 실행하는 LogUpdateSet의 새 코드다.

In [ ]:
    def LogUpdateSetABC(self, data):
        xs = data
        n = len(xs)
        
        # 샘플 통계 계산
        m = numpy.mean(xs)
        s = numpy.std(xs)
        
        for hypo in sorted(self.Values()):
            mu, sigma = hypo
            
            # 주어진 hypo에 대해 m의 로그 우도를 계산
            
            stderr_m = sigma / math.sqrt(n)
            loglike = EvalGaussianLogPdf(m, mu, stderr_m)
            
            # 주어진 hypo에 대해 s의 로그 우도 계산
            
            stderr_s = sigma / math.sqrt(2*(n-1))
            loglike += EvalGaussianLogPdf(s, sigma, stderr_s)
            
            self.Incr(hypo, loglike)

### (9) 로버스트 추정

데이터셋에는 오류일 가능성이 많은 아웃라이어가 많이 있다. <br />
예를 들어, 키가 61cm로 기록된 성인 세 명이 있는데, 이는 아마도 세상에서 가장 단신인 성인일 것이다. <br />
반면에, 키가 229cm로 기록된 여성이 네 명 있는데, 이는 세상에서 가장 장신인 여성들일 것이다. <br />
이 값들이 사실일 가능성도 있기는 하지만, 이런 극단적인 값은 변이 추정에 있어서 불균형적 영향을 미치기 때문에 바로잡아야 한다.

ABC는 요약 통계 기반이므로 전체 데이터셋을 다루는 것에 비해 요약 통계를 선택하는 것이 더 아웃라이어에 대한 신뢰도를 높인다. <br />
예를 들어, 샘플의 평균과 표준편차 대신, 중간값과 25번째와 75번째 백분위수의 차잇값인 4분위 범위를 사용할 수 있다. 

좀 더 일반적으로 분포 p의 주어진 어느 범위 내의 차잇값을 구하는 분위 내 범위를 계산할 수 있다.

In [ ]:
def MedianIPR(xs, p):
    cdf = thinkbayes2.MakeCdfFromList(xs)
    median = cdf.Percentile(50)
    
    alpha = (1-p) / 2
    ipr = cdf.Value(1-alpha) - cdf.Value(alpha)
    return median, ipr

xs는 연속값이다. p는 필요한 범위로, 예를 들자면 p = 0.5는 4분위 범위가 된다. <br />
MedianIPR은 xs의 CDF를 계산해서 중간값과 두 분위 수의 차이를 찾는다.<br />

가우시안 CDF를 사용해서 ipr을 sigma의 추정값으로 변환해서 표준편차의 주어진 수로 주어진 범위 내의 분포를 계산한다. <br />
예를 들어, 가우시안 분포의 68%가 표준편차 1 이내의 중간에 차지하고, 양쪽 끝에 16%가 있다는 잘 알려진 경험적 법칙이다. <br />
만약 16번째와 84번째 백분위값 사이의 범위를 계산한다면 결과가 2*sigma가 나온다는 것을 알 수 있다. <br /> 
따라서 68% IPR을 계산한 후 2로 나누는 식으로 sigma를 계산할 수 있다.

보다 일반적으로 sigmas의 어떤 숫자든 사용하게 할 수 있다. MedianS는 이런 더 일반적인 방식으로 계산을 수행한다.

In [ ]:
    def MedianS(xs, num_sigmas):
        half_p = thinkbayes2.StandardGaussuian(num_sigmas) - 0.5
        
        median, ipr = MedianIPR(xs, half_p*2)
        s = ipr / 2 / num_sigmas
        
        return median, s

xs는 값의 연속형이고, num_sigmas는 기반이 되는 결과의 표준편차 숫자다. <br />
결과는 mu를 추정하는 median과 sigma를 추정하는 s다.

마지막으로 LogUpdateSetABC에서 샘플 평균과 표준편차를 median과 S로 치환할 수 있다. <br />

mu와 sigma를 추정하기 위해 관측 백분위수를 사용하는게 좀 이상하다고 느껴질 수 있겠지만 <br />
이는 베이지안 접근법의 유연성을 설명하기 위한 예가 될 수 있다. 예를 들어, 이렇게 질문 할 수 있다. <br />
"mu와 sigma에 대한 가설 값과 오차가 나타내는 샘플링 프로세스가 주어졌을 때 샘플 통계의 주어진 집합을 만드는 우도는 얼마일까?"

우리는 원하는 샘플 통계를 어떤 것이든 사용할 수 있다. <br />
mu와 sigma의 위치와 분포의 퍼진 정도를 결정하므로 이런 성격을 가진 통계값을 선택해야 한다. <br />
예를 들어, 49번째와 51번째 백분위 수를 선택한다면 분포의 퍼짐 정도에 대한 정보는 거의 얻을 수 없으므로 <br />
데이터에 대해 상대적으로 제한을 받지 않는 sigma값을 추정할 때 사용한다. <br /> 
sigma의 모든 값은 관측값을 만드는 때에 거의 동일한 우도를 가지므로 sigma의 사후 본포는 사전 분포와 상당히 유사하다.

### (10) 누가 더 변이성이 높은가?

그래서 남자와 여자 중 누가 더 변이 계수가 큰가?

num_sigmas = 1 일 때 중간값과 IPR기반의 ABC를 사용해서, mu와 sigma 기반의 사후 결합 분포를 계산한다.<br />
다음의 다이어그램들은 x축으로 mu를 놓고, y축으로 sigma를 놓고, z축에 확률을 놓은 등고선 그래프를 보여준다.

미국 남성의 키의 평균과 표준편차에 대한 사후 결합 분포의 등고선

<img src="./Images/5.png" width=400 />

미국 여성의 키의 평균과 표준편차에 대한 사후 결합 분포의 등고선

<img src="./Images/6.png" width=400 />

각 결합 분포에 대해 CV의 사후 분포를 계산했다. 아래의 다이어그램은 남녀에 대해 이런 분포를 보여준다. <br />
남성의 평균은 0.0410, 여성의 평균은 0.0429다. <br /> 
이 분포 간에 겹치는 부분이 없으므로 여성이 키에 있어서는 남성보다 변이성이 더 높다는 결론을 확실하게 낼 수 있다. 

하지만 이 결과는 분위 내 범위 선택에 따라 달라진다는 것으로 밝혀졌다. <br />
num_sigmas = 1 일때는 여성이 더 변이성이 높다는 결론을 얻을 수 있지만, <br />
num_sigmas = 2의 경우 남성이 더 변이성이 높다는 가설이 동일한 신뢰도를 가진다.

이런 차이가 발생하는 원인은 작은 쪽에 치우친 경우가 남성이 더 많고 따라서 평균으로 부터의 거리도 커진다. <br />
따라서 변이 가설 평가는 변이 해석에 따라 달라진다. <br />
num_sigmas = 1 일 때 사람들이 평균 부근에 많다는 점에 집중하자. <br />
num_sigmas가 증가할수록, 극단값의 비중이 높아진다. <br />

무엇을 강조하는 것이 적합하는지 결정하기 위해서는 가설을 더욱 정확히 파악하는 것이 필요하다.

### (11) 토의

ABC에 대해서 두 가지 방식으로 생각해 볼 수 있다. <br />
첫 번째 해석은 이름의 의미대로, 원값보다 빠르게 계산하기 위해 사용하는 추정값이다. <br />
하지만 베이지안 분석이 항상 모델링 기반임을 떠올려 보면, 어차피 정확한 해답은 없다는 뜻이다. <br />
많은 흥미로운 물리 시스템은 여러 가능한 모델을 갖고, 각 모델은 서로 다른 결과를 낸다. <br />
이 결과를 해석하기 위해 모델을 평가한다.

ABC의 다른 해석으로는 "주어진 가설하에서 데이터의 우도는 어떻게 되는가?"라는 질문에 대한 대답으로 <br />
p(D|H)계산을 위한 위도의 대안 모델이라는 것이다. <br />
큰 데이터셋의 경우, 데이터의 우도는 매우 작아지고, 이는 제대로 된 질문을 할 수 없을 수도 있다는 것을 알려준다. <br />
우리가 정말로 알고자 하는 것은 데이터 같은 결과에 대한 우도로, 이 때 "같은"의 정의는 다른 모델을 결정하는 것일 수도 있다.

ABC의 기저에 깔린 개념은 두 데이터셋이 동일한 요약 통계치를 가질 때는 거의 동일하다는 것이다.

$ { x }_{ 1 } $